<a href="https://colab.research.google.com/github/salehihaniyeh/LLM/blob/main/Llama_2_FineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U autotrain-advanced
!pip install -U huggingface_hub

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!autotrain setup --update-torch

INFO     | 2024-05-01 21:47:04 | autotrain.cli.run_setup:run:43 - Installing latest xformers
INFO     | 2024-05-01 21:47:04 | autotrain.cli.run_setup:run:45 - Successfully installed latest xformers
INFO     | 2024-05-01 21:47:04 | autotrain.cli.run_setup:run:51 - Installing latest PyTorch
INFO     | 2024-05-01 21:47:10 | autotrain.cli.run_setup:run:53 - Successfully installed latest PyTorch


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!autotrain llm --train\
--project-name 'Apple-Issue-Prompt'\
--model TinyPixel/Llama-2-7B-bf16-sharded\
--data-path .\
--text-column text\
--use-peft\
--quantization int4 \
--lr 2e-4\
--train-batch-size 4\
--epochs 8\
--trainer sft > training.log &

In [4]:
import re

def extract_train_loss(log_file):
    """
    Extracts the training loss from the given log file.

    Args:
        log_file (str): Path to the log file.

    Returns:
        list: List of training loss values.
    """
    train_loss_values = []
    with open(log_file, 'r') as file:
        for line in file:
            match = re.search(r"'train_loss': (\d+\.\d+)", line)
            if match:
                train_loss = float(match.group(1))
                train_loss_values.append(train_loss)
    return train_loss_values

# Example usage:
log_file_path = 'training.log'
train_loss_values = extract_train_loss(log_file_path)
print("Training Loss Values:", float(train_loss_values[0]))

Training Loss Values: 0.4917277991771698


In [ ]:
import subprocess

# Define range of epochs to iterate through
min_epochs = 10
max_epochs = 21

# Dictionary to store training loss for each epoch
training_loss = {}

# Iterate through epochs
for epoch in range(min_epochs, max_epochs , 5):
    # Construct command with current number of epochs
    !autotrain llm --train\
    --project-name 'Apple-Issue-Prompt'\
    --model TinyPixel/Llama-2-7B-bf16-sharded\
    --data-path .\
    --text-column text\
    --use-peft\
    --quantization int4 \
    --lr 2e-4\
    --train-batch-size 4\
    --epochs {epoch}\
    --trainer sft > training.log &

    training_loss[epoch] = extract_train_loss('training.log')

print(f"Optimal number of epochs: {training_loss[0]}")

In [ ]:
!autotrain llm  --help

In [1]:
import peft
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.nn import DataParallel

tokenizer = AutoTokenizer.from_pretrained("/content/Apple-Issue-Prompt")
model = AutoModelForCausalLM.from_pretrained("/content/Apple-Issue-Prompt")



input_context = '''
###Human:
Generate a detailed description prompt for apple watch screen blinking.

###Assistant:
'''
input_ids = tokenizer.encode(input_context, return_tensors="pt")
output = model.generate(input_ids, min_length = 80, max_length=120, do_sample=True, temperature=0.3, num_return_sequences=1)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]


###Human:
Generate a detailed description prompt for apple watch screen blinking.

###Assistant:
Customer's Apple Watch Series 5 purchased in November 2021 is experiencing the screen blinking randomly. They reside in Richmond and have Xfinity as their internet provider. They have tried restarting the watch and updating to the latest software, but the issue persists.


In [2]:
input_context = '''
###Human:
Generate a detailed description prompt for icloud on macbook is not syncing.

###Assistant:
'''
input_ids = tokenizer.encode(input_context, return_tensors="pt")
output = model.generate(input_ids, min_length = 80, max_length=120, do_sample=True, temperature=0.3, num_return_sequences=1)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


###Human:
Generate a detailed description prompt for icloud on macbook is not syncing.

###Assistant:
Customer's iCloud on their MacBook Pro 13-inch purchased in July 2021 is not syncing. They are located in Honolulu and have Hawaiian Telcom as their internet provider. They have tried signing out and back into iCloud, but the issue persists.
